In [26]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

In [27]:
import random
import networkx as nx

def fast_connected_gnm(n, m, directed=False):
    if directed:
        raise ValueError("Directed connected G(n,m) is not well-defined for weak connectivity.")

    G = nx.Graph()
    G.add_nodes_from(range(n))

    # Step 1: random spanning tree (ensures connectivity)
    nodes = list(range(n))
    random.shuffle(nodes)
    for i in range(n-1):
        G.add_edge(nodes[i], nodes[i+1])

    # Step 2: add random edges until reaching m total edges
    needed = m - (n - 1)

    while G.number_of_edges() < m:
        u = random.randint(0, n-1)
        v = random.randint(0, n-1)
        if u != v and not G.has_edge(u, v):
            G.add_edge(u, v)

    return G

G = fast_connected_gnm(4783, 11140)


In [28]:
print(G)

Graph with 4783 nodes and 11140 edges
